In [1]:

# Code adapted from https://learnopencv.com/getting-started-with-pytorch-lightning/

import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import MNIST
from torchvision import transforms
import pytorch_lightning as pl
import os


### Defining the model object

Pytorch-lightning will handle everything we need to do, we just need to define the functions.


In [4]:
class model(pl.LightningModule):
    def __init__(self):
        super(model, self).__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1,28,kernel_size=5),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2))
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(28,10,kernel_size=2),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2))
        self.dropout1=torch.nn.Dropout(0.25)
        self.fc1=torch.nn.Linear(250,18)
        self.dropout2=torch.nn.Dropout(0.08)
        self.fc2=torch.nn.Linear(18,10)

 
    #This contains the manupulation on data that needs to be done only once such as downloading it
    def prepare_data(self):
        MNIST(os.getcwd(), train=True, download =True)
        MNIST(os.getcwd(), train=False, download =True)
    
    def train_dataloader(self):
        #This is an essential function. Needs to be included in the code
        #See here i have set download to false as it is already downloaded in prepare_data
        mnist_train=MNIST(os.getcwd(), train=True, download =False,transform=transforms.ToTensor())
        
        #Dividing into validation and training set
        self.train_set, self.val_set= random_split(mnist_train,[55000,5000])
        
        return DataLoader(self.train_set,batch_size=128)
        
    def val_dataloader(self):
        # OPTIONAL
        return DataLoader(self.val_set, batch_size=128)

    def test_dataloader(self):
        # OPTIONAL
        return DataLoader(MNIST(os.getcwd(), train=False, download=False, transform=transforms.ToTensor()), batch_size=128)


    def forward(self,x):
          x=self.layer1(x)
          x=self.layer2(x)
          x=self.dropout1(x)
          x=torch.relu(self.fc1(x.view(x.size(0), -1)))
          x=F.leaky_relu(self.dropout2(x))
          
          return F.softmax(self.fc2(x))

    def configure_optimizers(self):
        # Essential fuction
        #we are using Adam optimizer for our model
        return torch.optim.Adam(self.parameters())

    def training_step(self,batch,batch_idx):
          
          #extracting input and output from the batch
          x,labels=batch
          
          #doing a forward pass
          pred=self.forward(x)
          
          #calculating the loss
          loss = F.nll_loss(pred, labels)
          
          #logs
          logs={"train_loss": loss}
          
          output={
              #REQUIRED: It ie required for us to return "loss"
              "loss": loss,
              #optional for logging purposes
              "log": logs
          }
          
          return output


#abstracts the training, val and test loops

#max 40 epochs, full list of arguments: https://pytorch-lightning.readthedocs.io/en/latest/trainer.html
myTrainer=pl.Trainer(max_epochs=2)

model = model()
myTrainer.fit(model)



GPU available: False, used: False
TPU available: None, using: 0 TPU cores

  | Name     | Type       | Params
----------------------------------------
0 | layer1   | Sequential | 728   
1 | layer2   | Sequential | 1.1 K 
2 | dropout1 | Dropout    | 0     
3 | fc1      | Linear     | 4.5 K 
4 | dropout2 | Dropout    | 0     
5 | fc2      | Linear     | 190   
----------------------------------------
6.6 K     Trainable params
0         Non-trainable params
6.6 K     Total params


Epoch 0:   0%|          | 1/430 [00:00<00:38, 11.08it/s, loss=-0.0996, v_num=7]

<ipython-input-4-f1db27ad14e5>:49: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(self.fc2(x))


Epoch 0:  14%|█▍        | 62/430 [00:03<00:21, 17.00it/s, loss=-0.291, v_num=7]


/Users/mattsonthieme/miniconda3/envs/pytorch/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


1